In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import logging
logging.basicConfig(level=logging.INFO)

import warnings
warnings.simplefilter("ignore", ResourceWarning)

# Vectrix Demo 👨🏻‍💻
This notebook demonstrates the functions for importing data from various sources. 
Loading it into a VectorStore, and then using it to answer questions with a Retrieval Augemented Reasoning  🦜🔗 LangGraph.

## Importing Data
### 1. From a URL 🔗

**Web Crawling and Data Extraction Example**

This cell demonstrates the process of crawling a website, chunking the extracted content, and performing Named Entity Recognition (NER) using the Vectrix library.

#### Steps:

1. **Web Crawling**: 
   - Uses `vectrix.Crawler` to extract pages from "https://vectrix.ai"
   - Limits the crawl to a maximum of 20 pages

2. **Content Chunking**:
   - Utilizes `vectrix.Webchunker` to break down the extracted content
   - Chunks are created with a size of 500 characters and an overlap of 50 characters

3. **Named Entity Recognition**:
   - Employs `vectrix.Extract` with the 'Replicate' model
   - Uses the 'meta/meta-llama-3-70b-instruct' model for entity extraction

This example showcases a typical workflow for web data extraction and processing using the Vectrix library.


In [2]:
# Extracting data from a URL
from vectrix.importers import WebCrawler, ChunkData

crawler =  WebCrawler("https://www.elmos.be")
extracted_pages = crawler.extract()

print(f"Extracted {len(extracted_pages)} pages")

# Chunk the data

chunked_webdata = ChunkData.chunk_content(extracted_pages, chunk_size=500, chunk_overlap=50)

print("Example CHUNK: ")
print(chunked_webdata[1])

INFO:vectrix.importers.webscraper:Starting extraction pipeline for site: elmos.be
Fetching pages: 100%|##########| 1/1 [00:00<00:00, 10.09it/s]
/Users/ben/GitHub/vectrix/.venv/lib/python3.12/site-packages/bs4/builder/_htmlparser.py:134: ResourceWarning: unclosed <socket.socket fd=81, family=2, type=1, proto=6, laddr=('10.52.13.79', 60598), raddr=('5.134.4.204', 443)>
  attrvalue = '""'
/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/selector_events.py:879: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=81>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
INFO:vectrix.importers.webscraper:Visiting the following links: ['https://www.elmos.be/contacteer-ons', 'https://www.elmos.be/vind-it-talent', 'https://www.elmos.be/content-hub', 'https://www.elmos.be', 'https://www.elmos.be/opleidingen/masterclasses-voor-iters', 'https://www.elmos.be/over-ons/het-team-van-elmos-it-consultancy', 'https://www

KeyError: 'description'

In [ ]:
extracted_pages

In [ ]:
from vectrix import Weaviate

weaviate = Weaviate()
weaviate.set_colleciton("Vectrix")

response = weaviate.list_metadata()
print(response)

#### Adding Data to Vector Store using Weaviate

This code demonstrates how to add extracted web data to a Weaviate vector store using the Vectrix library.

**Steps**:

1. **Import and Instantiate Weaviate**:
   - Import necessary modules from Vectrix
   - Create a Weaviate instance

2. **Create a Collection**:
   - Set up a new collection named 'Vectrix'
   - Use a local Ollama model for embeddings
   - Specify the model name and URL

3. **Prepare Data for Vectorization**:
   - Create `VectorDocument` objects for each extracted result
   - Include metadata such as title, URL, content, type, and NER information

4. **Add Data to Vector Store**:
   - Use `weaviate.add_data()` to insert the prepared documents

5. **Perform a Test Query**:
   - Get a retriever from the Weaviate instance
   - Execute a sample query to verify functionality

**Note:**
- You can remove a collection using `weaviate.remove_collection("Vectrix")`
- This example uses a local Ollama model, but you can use any compatible embedding model

This code showcases the process of storing and querying vectorized web data using Weaviate and Vectrix, enabling efficient semantic search and retrieval.


In [ ]:
from vectrix.db import Weaviate

# Instantiation of the Vector store
weaviate = Weaviate()

In [ ]:
from vectrix.db import Weaviate

# Instantiation of the Vector store
weaviate = Weaviate()

weaviate.remove_collection("Vectrix")

# Create a new collection to store the documents (you can also append them to an existsing one)
weaviate.create_collection(name='Vectrix', 
                           embedding_model='Ollama', # For this example we use a local Ollama model, but you can use any other model 
                           model_name="mxbai-embed-large:335m",
                           model_url="http://host.docker.internal:11434") # Ollama Embeddings model URL


weaviate.add_data(chunked_webdata)


# Perform a quick search to see if it works
retriever = weaviate.get_retriever()
retriever.invoke('Who are the Vectrix founders ?')

In [ ]:
weaviate.set_colleciton('Vectrix')
retriever = weaviate.get_retriever()
retriever.invoke('Who are the Vectrix founders ?')

### From OneDrive ☁
We can also connect with a OneDrive folder and download the information from there. After this process we can add the information to our Vector store in the same way. This way we can extract various kinds of documents like PDF, World, PowerPoint ...

Downloads files from OneDrive, processes them with Unstructured, and stores in Weaviate.

Steps:
1. Connect to OneDrive using Azure credentials
2. List and download all files
3. Process files with Unstructured importer
4. Add processed documents to Weaviate vector store
5. Close OneDrive connection

Requirements:
- vectrix library (OneDrive connector, Unstructured importer)
- Weaviate client
- Azure credentials as environment variables (AZURE_CLIENT_ID, AZURE_CLIENT_SECRET)

Note: Close OneDrive connection before uploading to vector store.

In [ ]:
from vectrix.connectors.onedrive import OneDrive
from vectrix.importers.unstructured import Unstructured

drive = OneDrive(azure_client_id = os.environ.get('AZURE_CLIENT_ID'), azure_client_secret = os.environ.get('AZURE_CLIENT_SECRET'))

drive.list_files()

In [ ]:
# Download all files in the drive, returns a list of downloaded files
downloaded_files = drive.download_files()
print(downloaded_files)

# Process teh documents using unstructured (we can't load RAW PDF files into a Vector store)
importer = Unstructured()
documents = importer.process_files(downloaded_files)

# Add the documents to the Vector store
weaviate = Weaviate()
weaviate.set_colleciton('Vectrix')
weaviate.add_data(documents)

# Close the drive, do this BEFORE you want to upload the files to a vector store
drive.close()